<a href="https://colab.research.google.com/github/fcoliveira-utfpr/agrometeorologia/blob/main/05_bh_climatologico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inserindo informações**
---

In [2]:
# ============================================================
municipio = "Dourados" ###### Nome aqui entre aspas

estado = "MS" ###### Sigla do estado entre aspas

# ============================================================
# ============================================================

#importando bibliotecas
import numpy as np
import pandas as pd

# **Carrega base climática TerraClimate 1991-2020**
---

In [11]:
# Carrega base climática local
# ============================================================
url = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/TerraClimate_PR_PET_Normal_1991_2020.csv"
df = pd.read_csv(url)

# Separa colunas categóricas
cidade = df['Município']
uf = df['Estado']

# Converte valores com vírgula para ponto e numérico
df_valores = df.drop(columns=['Município', 'Estado'])
df_valores = df_valores.replace({',': '.'}, regex=True)
df_valores = df_valores.apply(pd.to_numeric, errors='coerce')

# Reconstrói o df
df = df_valores.copy()
df['Município'] = cidade
df['Estado'] = uf

# Filtra município e estado
df = df[(df['Município'] == municipio) & (df['Estado'] == estado)].copy()

# Correção: Alterar a lista de meses para corresponder ao formato das colunas (ex: 'PET_01')
meses = [f"{i:02d}" for i in range(1, 13)]

pet = [df[f"PET_{m}"].values[0] for m in meses]
chuva = [df[f"PR_{m}"].values[0] for m in meses]

mes = ["jan","fev","mar","abr","maio","jun","jul","ago","set","out","nov","dez"]

df_bh = pd.DataFrame(
    {
      'Meses': mes,
      'P (mm/mês)': chuva,
      'ETP (mm/mês)': pet,
    }
)
df_bh

,Meses,P (mm/mês),ETP (mm/mês)
0,jan,158.933333,133.543333
1,fev,152.400000,119.686667
2,mar,111.533333,123.853333
3,abr,111.600000,104.133333
4,maio,120.000000,85.313333
5,jun,75.333333,76.930000
6,jul,50.233333,88.060000
7,ago,46.233333,115.263333
8,set,101.400000,124.133333
9,out,170.566667,130.810000


# **balanço hídrico climatológico de Thothwaite-Mather**
---

In [33]:
# CAD
df_bh['CAD'] = 100

# Estado inicial de ARM (solo cheio na profundidade inicial)
ARM1 = [df_bh['CAD'].iloc[0]]

# P - ETP
df_bh['P-ETc'] = df_bh['P (mm/mês)'] - df_bh['ETP (mm/mês)']

# ARM
PETc1 = df_bh['P-ETc'].to_numpy()
CAD1 = df_bh['CAD'].to_numpy()

for p, cad in zip(PETc1, CAD1):
    prev = ARM1[-1]
    if p < 0:
        # Déficit: usa equação exponencial
        ARM1.append(prev * np.exp(p / cad))
    elif p + prev >= cad:
        # Encharcou: limita ao CAD
        ARM1.append(cad)
    else:
        # Situação normal
        ARM1.append(prev + p)
ARM1 = ARM1[1:]
df_bh['ARM (mm/mês)'] = ARM1

# ALT
ALT = [0]  # primeiro dia, variação zero
ALT += list(np.array(ARM1[1:]) - np.array(ARM1[:-1]))
df_bh['ALT'] = ALT

# ETR
df_bh['ETR (mm/mês)'] = np.where(
    df_bh['P-ETc'] < 0,
    df_bh['P (mm/mês)'] + df_bh['ALT'].abs(),  # solo complementa a chuva
    df_bh['ETP (mm/mês)']                           # sem limitação hídrica
)

# DEF
df_bh['DEF (mm/mês)'] = df_bh['ETP (mm/mês)'] - df_bh['ETR (mm/mês)']

# DEF
df_bh['EXC (mm/mês)'] = np.where(
    df_bh['ARM (mm/mês)'] < df_bh['CAD'],
    0,
    df_bh['P-ETc'] - df_bh['ALT']
)

df_bh

,Meses,P (mm/mês),ETP (mm/mês),P-ETc,CAD,ARM1,ARM,ARM (mm/mês),ALT,ETR,ETR (mm/mês),DEF (mm/mês),EXC (mm/mês)
0,jan,158.933333,133.543333,25.390000,100,100,100.000000,100.000000,0.000000,133.543333,133.543333,0.000000,25.390000
1,fev,152.400000,119.686667,32.713333,100,100,100.000000,100.000000,0.000000,119.686667,119.686667,0.000000,32.713333
2,mar,111.533333,123.853333,-12.320000,100,100,88.408683,88.408683,-11.591317,123.124651,123.124651,0.728683,0.000000
3,abr,111.600000,104.133333,7.466667,100,100,95.875349,95.875349,7.466667,104.133333,104.133333,0.000000,0.000000
4,maio,120.000000,85.313333,34.686667,100,100,100.000000,100.000000,4.124651,85.313333,85.313333,0.000000,30.562016
5,jun,75.333333,76.930000,-1.596667,100,100,98.416012,98.416012,-1.583988,76.917321,76.917321,0.012679,0.000000
6,jul,50.233333,88.060000,-37.826667,100,100,67.419673,67.419673,-30.996340,81.229673,81.229673,6.830327,0.000000
7,ago,46.233333,115.263333,-69.030000,100,100,33.805951,33.805951,-33.613722,79.847055,79.847055,35.416278,0.000000
8,set,101.400000,124.133333,-22.733333,100,100,26.931686,26.931686,-6.874265,108.274265,108.274265,15.859069,0.000000
9,out,170.566667,130.810000,39.756667,100,100,66.688353,66.688353,39.756667,130.810000,130.810000,0.000000,0.000000
